# KMeans

In [0]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

In [0]:
n_rows = 1000000
n_cols = 500
n_clusters_data = 400
cluster_std = 1.0
dtype='float32'
from sklearn.datasets import make_blobs
data, _ = make_blobs(
        n_samples=n_rows, n_features=n_cols, centers=n_clusters_data, cluster_std=cluster_std, random_state=0
    )  # make_blobs creates a random dataset of isotropic gaussian blobs.

data = data.astype(dtype)

### Convert dataset to Spark DataFrame

In [0]:
pd_data = pd.DataFrame({"features": list(data)})
df = spark.createDataFrame(pd_data)
df.write.mode('overwrite').parquet("/tmp/kmeans_notebook_data.parquet")
df = spark.read.parquet("/tmp/kmeans_notebook_data.parquet")

### We will use this function to build both the Spark RAPIDS ML (GPU) and Spark ML (CPU) linear estimator objects, demonstrating the common API

In [0]:
def build_kmeans_estimator(estimator_class):
    return ( 
            estimator_class()
            .setTol(1.0e-20)
            .setK(200)
            .setFeaturesCol("features")
            .setMaxIter(15)
           )

## Spark RAPIDS ML (GPU)

In [0]:
from spark_rapids_ml.clustering import KMeans
gpu_kmeans = build_kmeans_estimator(KMeans)

Estimator can be persisted and reloaded.

In [0]:
estimator_path = "/tmp/kmeans-estimator"

In [0]:
gpu_kmeans.write().overwrite().save(estimator_path)
gpu_kmeans_loaded = KMeans.load(estimator_path)

### Fit

In [0]:
start_time = time.time()
gpu_model = gpu_kmeans_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 25.13299584388733 sec

In [0]:
gpu_kmeans_loaded.getK()

Out[9]: 200

In [0]:
sorted_clusters = sorted([vec.tolist() for vec in gpu_model.clusterCenters()])

In [0]:
[vec[0:10] for vec in sorted_clusters[0:2]]

Out[11]: [[-10.011515617370605,
 -7.989240646362305,
 -9.93497371673584,
 4.296850204467773,
 2.279391288757324,
 -4.386820316314697,
 7.950189113616943,
 -0.90474534034729,
 -8.987994194030762,
 7.664593696594238],
 [-9.888717651367188,
 -4.852360725402832,
 8.597984313964844,
 -8.74846363067627,
 1.81229829788208,
 2.888047933578491,
 9.357666969299316,
 -5.18804407119751,
 -6.063205242156982,
 6.238801956176758]]

### Transform

In [0]:
model_path = "/tmp/kmeans-model"

In [0]:
gpu_model.write().overwrite().save(model_path)

In [0]:
gpu_model_loaded = gpu_model.read().load(model_path)

In [0]:
[vec[0:10] for vec in sorted(gpu_model_loaded.cluster_centers_)[0:2]]

Out[15]: [[-10.011515617370605,
 -7.989240646362305,
 -9.93497371673584,
 4.296850204467773,
 2.279391288757324,
 -4.386820316314697,
 7.950189113616943,
 -0.90474534034729,
 -8.987994194030762,
 7.664593696594238],
 [-9.888717651367188,
 -4.852360725402832,
 8.597984313964844,
 -8.74846363067627,
 1.81229829788208,
 2.888047933578491,
 9.357666969299316,
 -5.18804407119751,
 -6.063205242156982,
 6.238801956176758]]

In [0]:
transformed_df = gpu_model_loaded.setPredictionCol("transformed").transform(df)

In [0]:
transformed_df.printSchema()

root
-- features: array (nullable = true)
 |-- element: float (containsNull = true)
-- transformed: integer (nullable = true)

In [0]:
transformed_df.count()

Out[18]: 1000000

In [0]:
transformed_df.show(10)

+--------------------+-----------+
 features|transformed|
+--------------------+-----------+
[-7.445706367, -5...| 20|
[0.379327744, -2....| 13|
[6.53111887, -4.3...| 167|
[4.252857685, 0.2...| 20|
[-0.321669161, -2...| 15|
[6.502286434, 1.7...| 119|
[5.633553028, 7.9...| 113|
[-8.306350708, -9...| 183|
[-1.546614289, -7...| 106|
[1.793183923, -8....| 81|
+--------------------+-----------+
only showing top 10 rows

## Spark ML (CPU)

In [0]:
from pyspark.ml.clustering import KMeans
cpu_kmeans = build_kmeans_estimator(KMeans)

Convert array sql type to VectorUDT Dataframe expected by Spark ML algos (Note: Spark RAPIDS ML also accepts VectorUDT Dataframes in addition to array type Dataframe above, along with a scalar column format - see docs).

In [0]:
from pyspark.ml.functions import array_to_vector

In [0]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

### Fit

In [0]:
start_time = time.time()
cpu_kmeans_model = cpu_kmeans.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 553.3065311908722 sec

In [0]:
type(cpu_kmeans_model.clusterCenters()[0])

Out[24]: numpy.ndarray

In [0]:
sorted_cpu_cluster_centers = sorted([vec.tolist() for vec in cpu_kmeans_model.clusterCenters()])
[vec[0:10] for vec in sorted_cpu_cluster_centers[0:2]]

Out[25]: [[-9.88871803264618,
 -4.852364576244354,
 8.597981192970277,
 -8.748464743041993,
 1.812296859824774,
 2.888048359696567,
 9.357668845748902,
 -5.188045910453797,
 -6.063206001377106,
 6.238802798461914],
 [-9.656762870025636,
 -6.5508945960998535,
 9.329511338615418,
 9.624343802261354,
 -9.630506665420533,
 6.184547395324707,
 -1.2715576951929748,
 6.077930819416046,
 -0.41267842022195694,
 8.600529252624511]]

### Transform

In [0]:
spark_transformed = cpu_kmeans_model.setPredictionCol("transformed").transform(vector_df)

In [0]:
spark_transformed.filter(spark_transformed.transformed >= 0).count()

Out[27]: 1000000

In [0]:
spark_transformed.show(10)

+--------------------+-----------+
 features|transformed|
+--------------------+-----------+
[-7.4457063674926...| 89|
[0.37932774424552...| 183|
[6.53111886978149...| 173|
[4.25285768508911...| 18|
[-0.3216691613197...| 79|
[6.50228643417358...| 7|
[5.63355302810668...| 85|
[-8.3063507080078...| 14|
[-1.5466142892837...| 101|
[1.79318392276763...| 173|
+--------------------+-----------+
only showing top 10 rows